In [34]:
from torchvision.transforms import transforms
#from preprocessing import Imageloader, custom_metrics, data_split


transform = transforms.Compose(
        [transforms.ToPILImage(),
         transforms.Grayscale(num_output_channels=3),
         transforms.Resize((224, 224)),
         transforms.ToTensor(),
         transforms.Normalize((0.5), (0.5)),
         ]
    )
annotations_file = "/scratch/snx3000/rnef/Chest/allLabels.csv"
img_dir = "/scratch/snx3000/rnef/Chest"
model_dir = "../models/"

dataset = Imageloader(annotations_file, img_dir, transform)


In [35]:
splitter = data_split(dataset)
test_set,train_set = splitter.splitter()


In [36]:
len(test_set)

73059

max(test_set)

In [37]:
len(dataset)

91323

In [38]:
import torch
import torch.nn as nn
x = nn.Sigmoid()
loss = nn.BCELoss()
input_prob = torch.randn(4, requires_grad=True)
target_prob = torch.empty(4).random_(3)
output_prob = loss(x(input_prob), target_prob)
output_prob.backward()
print(output_prob)

tensor(1.2414, grad_fn=<BinaryCrossEntropyBackward0>)


### Ciafar testing

In [39]:
import pandas as pd
annotations_file = "/scratch/snx3000/rnef/Chest/sampleLabels.csv"
df_123 = pd.read_csv(annotations_file).iloc[1:6, 1]
df_123

1    0
2    0
3    0
4    0
5    0
Name: category, dtype: int64

In [6]:
from torch.utils.data import DataLoader, SubsetRandomSampler
import numpy as np

class data_split:
    def __init__(self, data):
        self.data = data
        
        
    def splitter(self,validation_split = 0.2,shuffle_dataset = True):
        # Creating data indices for training and validation splits:
        dataset_size = len(self.data)
        indices = list(range(dataset_size))
        
        if shuffle_dataset:
            np.random.seed(402)
            np.random.shuffle(indices)
            
        split = int(np.floor(validation_split * dataset_size))
        train_indices, val_indices = indices[split:], indices[:split]

        # Creating data samplers
        train_sampler = SubsetRandomSampler(train_indices)
        valid_sampler = SubsetRandomSampler(val_indices)
        
        return  train_sampler, valid_sampler

In [7]:
from torch.utils.data.dataset import Dataset
import pandas as pd
class Imageloader(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0]) 
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, int(label)

In [13]:
from torchvision.transforms import transforms
import torch
annotations_file = "../data/Chest/samples100.csv"
img_dir = "../data/Chest/"
transform = transforms.Compose(
    [transforms.ToPILImage(),
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((224,224)),
    #transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5),(0.5)),
         ]
    )
dataset = Imageloader(annotations_file, img_dir, transform)

    #Split dataset into train and validation
splitt = data_split(dataset)
train_sampler, valid_sampler = splitt.splitter()


train_loader = torch.utils.data.DataLoader(dataset,
                                               batch_size=10,
                                               sampler=train_sampler
                                               )
    
test_loader = torch.utils.data.DataLoader(dataset,
                                              batch_size=10,
                                              sampler=valid_sampler
                                              )




In [14]:
#from abc import ABC
from PIL import Image
import torch
from torch.utils.data.dataset import Dataset
from tqdm import tqdm

import os
import pandas as pd
from torchvision.io import read_image
import numpy as np
from torch.utils.data import DataLoader, SubsetRandomSampler
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for epoch in range(2):
        print(f"*************Training*************** \n epoch {epoch + 1} from 10")
        total_train = 0.0
        correct_train = 0.0
        # training
        
        #batch_index, (faces, labels) in enumerate(train_loader)
        for i, data in enumerate(tqdm(train_loader, 0)):

            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            print(i)
            print(labels)

            # zero the parameter gradients
            #optimizer.zero_grad()

            # forward + backward + optimize
            
            #outputs = model(inputs)
            #print(outputs)
            #loss = criterion(outputs, labels)
            #loss.backward()
            #optimizer.step()

*************Training*************** 
 epoch 1 from 10


 12%|█▎        | 1/8 [00:00<00:02,  3.50it/s]

0
tensor([0, 0, 1, 0, 1, 0, 0, 0, 0, 1])


 25%|██▌       | 2/8 [00:00<00:01,  3.66it/s]

1
tensor([0, 0, 1, 0, 1, 1, 0, 1, 0, 1])


...

5
tensor([0, 0, 0, 0, 1, 0, 0, 0, 1, 1])


 88%|████████▊ | 7/8 [00:01<00:00,  3.73it/s]

6
tensor([1, 0, 0, 0, 1, 1, 0, 1, 1, 1])


100%|██████████| 8/8 [00:02<00:00,  3.79it/s]

7
tensor([1, 1, 0, 1, 1, 0, 1, 0, 1, 0])


In [15]:
for b in range(10):
    for i, data in enumerate(tqdm(test_loader, 0)):
            images_test, labels_test = data
            images_test, labels_test = images_test.to(device), labels_test.to(device)
            print(labels_test)

                

 50%|█████     | 1/2 [00:00<00:00,  3.70it/s]

tensor([1, 0, 1, 0, 1, 1, 0, 0, 1, 1])


100%|██████████| 2/2 [00:00<00:00,  3.50it/s]


tensor([1, 0, 0, 1, 1, 1, 0, 0, 0, 1])


 50%|█████     | 1/2 [00:00<00:00,  3.75it/s]

...

tensor([1, 1, 0, 0, 1, 1, 0, 1, 1, 1])


 50%|█████     | 1/2 [00:00<00:00,  4.36it/s]

tensor([1, 1, 1, 0, 0, 1, 0, 1, 1, 0])


100%|██████████| 2/2 [00:00<00:00,  4.10it/s]

tensor([0, 0, 1, 1, 1, 0, 0, 0, 1, 1])


In [32]:
shuffle_dataset = True
dataset_size = len(dataset)
indices = list(range(dataset_size))
        
if shuffle_dataset:
    np.random.seed(402)
    np.random.shuffle(indices)
    
    
split = int(np.floor(0.2 * dataset_size))
         
train_indices, val_indices = indices[split:], indices[:split]

        # Creating data samplers
#train_sampler = SubsetRandomSampler(train_indices)
#valid_sampler = SubsetRandomSampler(val_indices)
indices

[64,
 61,
 24,
 78,
 128,
 46,
 111,
 32,
 28,
 156,
 25,
 189,
 63,
 196,
 138,
 97,
 81,
 22,
 37,
 17,
 175,
 104,
 90,
 73,
 139,
 4,
 16,
 56,
 126,
 180,
 181,
 33,
 71,
 95,
 173,
 160,
 50,
 36,
 166,
 43,
 1,
 178,
 142,
 39,
 69,
 199,
 94,
 190,
 185,
 54,
 53,
 119,
 176,
 109,
 89,
 12,
 118,
 113,
 179,
 93,
 155,
 49,
 3,
 72,
 130,
 135,
 20,
 184,
 87,
 75,
 38,
 67,
 124,
 57,
 157,
 44,
 127,
 30,
 162,
 103,
 194,
 140,
 167,
 136,
 14,
 77,
 0,
 168,
 195,
 158,
 68,
 105,
 183,
 74,
 182,
 26,
 35,
 40,
 148,
 146,
 115,
 145,
 51,
 144,
 80,
 161,
 100,
 110,
 191,
 101,
 11,
 153,
 106,
 121,
 42,
 177,
 88,
 18,
 41,
 137,
 171,
 164,
 98,
 48,
 123,
 83,
 45,
 5,
 92,
 152,
 197,
 96,
 82,
 143,
 102,
 114,
 60,
 147,
 65,
 66,
 117,
 122,
 34,
 107,
 13,
 31,
 108,
 134,
 59,
 29,
 125,
 141,
 159,
 19,
 198,
 84,
 52,
 76,
 27,
 163,
 91,
 120,
 186,
 21,
 58,
 47,
 170,
 2,
 149,
 132,
 172,
 151,
 193,
 86,
 116,
 85,
 112,
 187,
 23,
 169,
 131,
 192,
 17

In [34]:
len(val_indices)

40